# Capstone Project 1: Data Wrangling 1

Data source: https://www.yelp.com/dataset. The first dataset used includes 192,609 businesses with over 1.2 million business attributes like hours, parking, availability, and ambience. The second dataset used includes the aggregated check-ins over time for each of the 192,609 businesses. (According to Yelp)

Data wrangling activities:
- Read json files.
- Format json strings.
- Create pandas dataframes from json strings.
- Filter businesses into restaurants dataframe.
- Filter restaurants outliers - those with less than 10 reviews.
- Triage restaurants into those of good, moderate and poor standing based on stars into 3 restaurant dataframes.
- Triage checkins by restaurants standing (good, moderate, poor) into 3 restaurant checkin dataframes.
- Parse and reshape date columns of restaurant checkin dataframes.
- Sort and reset index for all resulting dataframes before using for analysis.

Data wrangling result:
- 1 dataframe containing restaurants in good standing (4 or more stars).
- 1 dataframe containing restaurants in moderate standing (between 2 and 4 stars).
- 1 dataframe containing restaurants in poor standing (less than 2 stars).
- 1 dataframe containing checkins of restaurants in good standing with date column parsed and reshaped.
- 1 dataframe containing checkins of restaurants in moderate standing with date column parsed and reshaped.
- 1 dataframe containing checkins of restaurants in poor standing with date column parsed and reshaped.

In [1]:
import pandas as pd
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read file into list
with open('dataset/business.json','r',encoding='utf8') as f:
    businesses = f.readlines()
with open('dataset/checkin.json','r',encoding='utf8') as f:
    checkins = f.readlines()

# Remove the trailing "\n" from each line
businesses = list(map(lambda x: x.rstrip(), businesses))
checkins = list(map(lambda x: x.rstrip(), checkins))

# Prepare json string
bjson = '['+','.join(businesses)+']'
cjson = '['+','.join(checkins)+']'

# Load json string into pandas dataframe
dfb = pd.read_json(bjson)
dfc = pd.read_json(cjson)

In [2]:
# Create boolean array for filtering for Restaurants
isrest = dfb.categories.str.contains('Restaurants') & pd.notnull(dfb.categories)

# Filter the businesses dataframe with boolean array and assign to restaurants dataframe
restaurants = dfb[isrest]


In [3]:
# Exploratory data analysis
print(dfc.shape) # There is no checkin data for 30659 of the businesses
print(restaurants.shape) # 59371 out of the 192609 businesses are restaurants
print(restaurants.columns)

(161950, 2)
(59371, 14)
Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')


In [4]:
# Filter out restaurants with less than 10 reviews.
restaurants = restaurants[restaurants.review_count > 10]

# Triage restaurants into 3 categories of standing using star ratings. Good, moderate and poor. Assign to dataframes.
goodr = restaurants[restaurants.stars >= 4]
poorr = restaurants[restaurants.stars <= 2]
modr = restaurants[restaurants.stars < 4]
modr = modr[modr.stars > 2]

In [5]:
print(goodr.shape)
print(modr.shape)
print(poorr.shape)


(16156, 14)
(21233, 14)
(2241, 14)


In [6]:
# Create boolean arrays for filtering for restaurant checkins
isgoodrc = dfc['business_id'].isin(goodr['business_id'])
ismodrc = dfc['business_id'].isin(modr['business_id'])
ispoorrc = dfc['business_id'].isin(poorr['business_id'])

# Filter the checkins dataframe with boolean arrays and assign to new dataframes
goodrc = dfc[isgoodrc]
modrc = dfc[ismodrc]
poorrc = dfc[ispoorrc]

print(goodrc.shape)
print(modrc.shape)
print(poorrc.shape)

(16103, 2)
(21156, 2)
(2230, 2)


In [7]:
goodrc.head(4)

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
5,--9e1ONYQuAa-CB_Rrw7Tw,"2010-02-08 05:56:47, 2010-02-15 04:47:42, 2010..."
10,--FBCX-N37CMYDfs790Bnw,"2010-05-31 07:57:10, 2010-05-31 07:58:37, 2010..."
26,--cZ6Hhc9F7VkKXxHMVZSQ,"2010-04-24 19:48:58, 2010-05-01 19:28:23, 2010..."


In [8]:
# Import datetime parse date column in checkin dataframes
import datetime as dt

# Create function for parsing dates and reshaping checkin dataframe. 
# Returns new dataframe with one row for each checkin date of each restaurant.
def transform_df(df):
    rclist = []
    for idx,row in df.iterrows():
        dates = row['date'].split(',')
        for d in dates:
            rcdict = {}
            rcdict.update({'business_id':row['business_id'],'date':dt.datetime.strptime(d.strip(),'%Y-%m-%d %H:%M:%S')})
            rclist.append(rcdict)
    rcdf = pd.DataFrame(rclist)
    return rcdf

In [9]:
# Call function on each checkin dataframe and assign result to new dataframe.
goodr_checkins = transform_df(goodrc)
modr_checkins = transform_df(modrc)
poorr_checkins = transform_df(poorrc)

In [10]:
# Sort restaurants dataframes by stars in descending order and reset index
goodr = goodr.sort_values('stars',ascending=False).reset_index(drop=True)
modr = modr.sort_values('stars',ascending=False).reset_index(drop=True)
poorr = poorr.sort_values('stars',ascending=False).reset_index(drop=True)


In [16]:
goodr.head(10)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,1422 Dundas Street W,"{'BusinessParking': '{'garage': False, 'street...",tAzdE1tRcsznfWN6TOLP3Q,"American (Traditional), Diners, Breakfast & Br...",Toronto,"{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ...",1,43.649729,-79.429364,Egg Sunrise Grill,M6J 1Y5,15,5.0,ON
1,1712 E Broadway,"{'RestaurantsDelivery': 'True', 'WheelchairAcc...",eZqoDfBg-xOS8WdcZCYM2A,"Sandwiches, Smokehouse, Barbeque, American (Tr...",Phoenix,"{'Monday': '11:0-16:0', 'Tuesday': '11:0-14:0'...",1,33.406990,-112.044537,JL Smokehouse,85040,74,5.0,AZ
2,497 Warren C Coleman Blvd S,"{'RestaurantsPriceRange2': '1', 'NoiseLevel': ...",uo-40ZDaRJBoh-PY4dbCgQ,"Dominican, Restaurants, Spanish, Caribbean",Concord,"{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",0,35.383468,-80.584428,Pilón Dominican Kitchen,28025,18,5.0,NC
3,4409 Boulevard Saint-Laurent,"{'RestaurantsTableService': 'False', 'BikePark...",m08WjSdWsFtZdhQqrFqjAQ,"Active Life, Yoga, Brazilian, Restaurants, Bre...",Montréal,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:0', '...",1,45.519630,-73.584917,Belém Café & Yoga - L'Espace Café,H2W 1Z8,11,5.0,QC
4,5525 Avenue de Monkland,"{'Ambience': '{'touristy': False, 'hipster': F...",Jbj4VGceJVKXOoN0zQmFzw,"Desserts, Coffee & Tea, Restaurants, Food, Cafes",Montréal,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-21:0'...",1,45.476666,-73.622197,Dessert Cafe Momo,H4A 1C7,37,5.0,QC
5,3635 Las Vegas Blvd S,"{'WheelchairAccessible': 'True', 'BusinessAcce...",iBPyahdJRP5y0t25fF2W9w,"Hotels, Food, Event Planning & Services, Hotel...",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.114537,-115.172678,Lip Smacking Foodie Tours,89109,257,5.0,NV
6,609 S Trenton Ave,"{'Caters': 'False', 'BusinessParking': '{'gara...",C3D5ww46iTU6LCWgSwM0vA,"Bakeries, Cafes, Food, Restaurants",Wilkinsburg,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-18:0', '...",1,40.438958,-79.892178,Madeleine Bakery And Bistro,15221,34,5.0,PA
7,4729 W Olive Ave,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...",rNzfJ3rm6gqUnb0I_YfPcQ,"Donuts, Food, Restaurants",Glendale,"{'Monday': '4:0-13:0', 'Tuesday': '4:0-13:0', ...",0,33.567506,-112.160916,Vantastic Donuts & Bagels,85302,19,5.0,AZ
8,"2900 Westinghouse Blvd, Ste 116","{'Caters': 'True', 'RestaurantsTableService': ...",jRxLnJ9my31Re_O2MvQGhw,"Thai, Caribbean, Restaurants, Indian, Fast Foo...",Charlotte,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-16:0', ...",1,35.132820,-80.952997,The Tasty Bowl Subs N Such,28273,86,5.0,NC
9,5527 Verona Rd,"{'RestaurantsReservations': 'False', 'WiFi': '...",3DQKaQN2wjWTWUN0EBwkzw,"Restaurants, Pizza",Verona,None,1,40.482728,-79.837817,Della Sala's Pizza,15147,21,5.0,PA


In [13]:
%store goodr
%store modr
%store poorr
%store goodr_checkins
%store modr_checkins
%store poorr_checkins
%store restaurants

Stored 'goodr' (DataFrame)
Stored 'modr' (DataFrame)
Stored 'poorr' (DataFrame)
Stored 'goodr_checkins' (DataFrame)
Stored 'modr_checkins' (DataFrame)
Stored 'poorr_checkins' (DataFrame)
Stored 'restaurants' (DataFrame)


In [116]:
# Create user defined function to pivot out the attributes we want to analyze
import ast
def pivot_attributes(akeys,df):
    for i,r in df[df['attributes'].notna()].iterrows():
        # Assign the attributes column value to a dictionary
        attributes = r['attributes']
        for k in akeys:
            if k in attributes:
                if attributes[k][0] == 'u':
                    newval = pd.Series(attributes[k][2:-1],name=k,index=[i])
                elif attributes[k][0] == "'":
                    newval = pd.Series(attributes[k][1:-1],name=k,index=[i])
                else:
                    newval = pd.Series(attributes[k],name=k,index=[i])
                df.update(newval)
    return df

In [75]:
attribute_keys = ['RestaurantsReservations','WiFi','HasTV','RestaurantsDelivery','RestaurantsTakeout',
                  'RestaurantsGoodForGroups','OutdoorSeating','Alcohol','RestaurantsPriceRange2',
                  'GoodForKids','NoiseLevel','RestaurantsAttire']
for k in attribute_keys:
    goodr[k] = ''
    modr[k] = ''
    poorr[k] = ''

In [ ]:
goodr = pivot_attributes(attribute_keys,goodr)
goodr.head(1)